In [61]:
import os
import pycurl
import io

from html.parser import HTMLParser

In [192]:
cpage="https://www.allrecipes.com/search/results/?ingIncl=flour,sugar&sort=re&page="
class MyHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.recording = 0
        self.data = []
        
    def handle_starttag(self, tag, attrs):
        for name, value in attrs:
            if value == "grid-card-image-container":
                self.recording = 1
            if name == "href" and self.recording:
                #print("id:", name, "value:", value)
                self.data.append(value)
                self.recording = 0


In [411]:
parser = MyHTMLParser()

for i in range(50):
    #print("PAGE", i+1)
    url = cpage+str(i+1)
    print(url)
    buffer = io.BytesIO()
    c = pycurl.Curl()

    c.setopt(c.URL, url)
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()

    body = buffer.getvalue()
    html=body.decode('iso-8859-1')
    parser.feed(html)


https://www.allrecipes.com/search/results/?ingIncl=flour,sugar&sort=re&page=1


error: (56, 'Send failure: Connection was reset')

In [200]:
parser.data


['https://www.allrecipes.com/recipe/33176/delicious-gluten-free-pancakes/',
 'https://www.allrecipes.com/recipe/244812/peach-financiers/',
 'https://www.allrecipes.com/recipe/238387/chef-johns-blackberry-buckle/',
 'https://www.allrecipes.com/recipe/10687/mrs-siggs-snickerdoodles/',
 'https://www.allrecipes.com/recipe/8139/gluten-free-yellow-cake/',
 'https://www.allrecipes.com/recipe/265860/fresh-fruit-frangipane-tart/',
 'https://www.allrecipes.com/recipe/261757/orange-cake-with-semolina-and-almonds/',
 'https://www.allrecipes.com/recipe/246901/gluten-free-rhubarb-upside-down-cake/',
 'https://www.allrecipes.com/recipe/240496/gluten-free-sugar-cookies/',
 'https://www.allrecipes.com/recipe/244165/easy-almond-thin-cookies/',
 'https://www.allrecipes.com/recipe/246328/french-macarons/',
 'https://www.allrecipes.com/recipe/217895/fluffy-gluten-free-cornbread/',
 'https://www.allrecipes.com/recipe/9864/cracked-sugar-cookies-i/',
 'https://www.allrecipes.com/recipe/11624/snickerdoodles-i/

In [407]:
def fr_to_dec(fr):
    f_s = re.search('([0-9]\s*/\s*[0-9]+)', fr)
    f = 0.
    if f_s != None:
        myfrac = re.sub('\s','',f_s.group(0))
        f += float(eval(myfrac))
     
    fint_s = re.search('([0-9]+)', fr)
    f += float(fint_s.group(1))
    return(f)


In [409]:
import numpy as np
import pandas as pd

import re

from recipe_scrapers import scrape_me



df = pd.DataFrame({"url": parser.data})
df.at[:,'eggs_any'] = 0.

for i in range(len(parser.data)):
#for i in range(100):
    scraper = scrape_me(parser.data[i])
    
    print(i, parser.data[i])
    df.at[i, 'name'] = scraper.title()
    
    for string in scraper.ingredients():
        #print(string)
        amt = ''
        unit = ''
        ing = ''

        m = re.search('([0-9\s/]+)\s*(\w+)\s*([\s\w]+)', string)
        if m == None:
            continue
        matches = len(m.groups())
        
        if matches < 2:
            continue 
            
        amt = m.group(1)
        amt_isnum = re.search('([0-9])', amt) != None
        if amt_isnum == 0:
            continue
        
        #print(string)
        amt_n = fr_to_dec(amt)
        
        ing = m.group(3)
        unit = m.group(2)
        if ing == '':
            unit = ''
            ing = m.group(2)
            
        if re.match('.*egg[s?]\s*', string) != None:
            df.at[i, 'eggs_any'] = int(amt)
        elif re.match('.*cup.*', unit) != None:
            df.at[i, ing] = amt_n
        elif re.match('.*tablespoon.*', unit) != None:
            df.at[i, ing] = amt_n / 16
        elif re.match('.*teaspoon.*', unit) != None:
            df.at[i, ing] = amt_n / 48
        
        
        #df.at[i, ing] = amt+'='+unit

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))

In [405]:
df.head()


['https://www.allrecipes.com/recipe/33176/delicious-gluten-free-pancakes/',
 'https://www.allrecipes.com/recipe/244812/peach-financiers/',
 'https://www.allrecipes.com/recipe/238387/chef-johns-blackberry-buckle/',
 'https://www.allrecipes.com/recipe/10687/mrs-siggs-snickerdoodles/',
 'https://www.allrecipes.com/recipe/8139/gluten-free-yellow-cake/',
 'https://www.allrecipes.com/recipe/265860/fresh-fruit-frangipane-tart/',
 'https://www.allrecipes.com/recipe/261757/orange-cake-with-semolina-and-almonds/',
 'https://www.allrecipes.com/recipe/246901/gluten-free-rhubarb-upside-down-cake/',
 'https://www.allrecipes.com/recipe/240496/gluten-free-sugar-cookies/',
 'https://www.allrecipes.com/recipe/244165/easy-almond-thin-cookies/',
 'https://www.allrecipes.com/recipe/246328/french-macarons/',
 'https://www.allrecipes.com/recipe/217895/fluffy-gluten-free-cornbread/',
 'https://www.allrecipes.com/recipe/9864/cracked-sugar-cookies-i/',
 'https://www.allrecipes.com/recipe/11624/snickerdoodles-i/